# --- Day 19: Aplenty ---
The Elves of Gear Island are thankful for your help and send you on your way. They even have a hang glider that someone stole from Desert Island; since you're already going that direction, it would help them a lot if you would use it to get down there and return it to them.

As you reach the bottom of the relentless avalanche of machine parts, you discover that they're already forming a formidable heap. Don't worry, though - a group of Elves is already here organizing the parts, and they have a system.

To start, each part is rated in each of four categories:
```
x: Extremely cool looking
m: Musical (it makes a noise when you hit it)
a: Aerodynamic
s: Shiny
```
Then, each part is sent through a series of workflows that will ultimately accept or reject the part. Each workflow has a name and contains a list of rules; each rule specifies a condition and where to send the part if the condition is true. The first rule that matches the part being considered is applied immediately, and the part moves on to the destination described by the rule. (The last rule in each workflow has no condition and always applies if reached.)

Consider the workflow ex{x>10:one,m<20:two,a>30:R,A}. This workflow is named ex and contains four rules. If workflow ex were considering a specific part, it would perform the following steps in order:
```
Rule "x>10:one": If the part's x is more than 10, send the part to the workflow named one.
Rule "m<20:two": Otherwise, if the part's m is less than 20, send the part to the workflow named two.
Rule "a>30:R": Otherwise, if the part's a is more than 30, the part is immediately rejected (R).
Rule "A": Otherwise, because no other rules matched the part, the part is immediately accepted (A).
```
If a part is sent to another workflow, it immediately switches to the start of that workflow instead and never returns. If a part is accepted (sent to A) or rejected (sent to R), the part immediately stops any further processing.

The system works, but it's not keeping up with the torrent of weird metal shapes. The Elves ask if you can help sort a few parts and give you the list of workflows and some part ratings (your puzzle input). For example:
```
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
```
The workflows are listed first, followed by a blank line, then the ratings of the parts the Elves would like you to sort. All parts begin in the workflow named in. In this example, the five listed parts go through the following workflows:
```
{x=787,m=2655,a=1222,s=2876}: in -> qqz -> qs -> lnx -> A
{x=1679,m=44,a=2067,s=496}: in -> px -> rfg -> gd -> R
{x=2036,m=264,a=79,s=2244}: in -> qqz -> hdj -> pv -> A
{x=2461,m=1339,a=466,s=291}: in -> px -> qkq -> crn -> R
{x=2127,m=1623,a=2188,s=1013}: in -> px -> rfg -> A
```
Ultimately, three parts are accepted. Adding up the x, m, a, and s rating for each of the accepted parts gives 7540 for the part with x=787, 4623 for the part with x=2036, and 6951 for the part with x=2127. Adding all of the ratings for all of the accepted parts gives the sum total of 19114.

**Sort through all of the parts you've been given; what do you get if you add together all of the rating numbers for all of the parts that ultimately get accepted?**

In [1]:
from utilities import get_lines

In [2]:
lines = get_lines('input')

In [3]:
blank_index = lines.index('')
wf_lines = lines[:blank_index]
parts_lines = lines[blank_index+1:]

In [4]:
len(wf_lines), len(parts_lines)

(576, 200)

In [5]:
class Rule:
    '''the Rule has a condition and next workflow'''
    def __init__(self, rule):
        self.rule = rule
        self.category = self.rule[0]
        self.operation = self.rule[1]
        self.rating = int(self.rule[2:].split(':')[0])
        self.next_workflow = self.rule.split(':')[1]

    def __repr__(self):
            return self.rule

In [6]:
wf_lines[0]

'ft{m<1347:R,m<1967:A,A}'

In [7]:
Rule('a<2006:qkq').rating

2006

In [8]:
class Workflow:
    def __init__(self, line):
        self.workflow = line
        self.name = line.split('{')[0]
        self.rules = line.split('{')[1].split('}')[0].split(',')[0:-1]
        self.next_workflow = line.split(',')[-1].split('}')[0]

        def _parse_rules(self):
            parsed_rules = list()
            for rule in self.rules:
                parsed_rules.append(Rule(rule))
            self.rules = parsed_rules
            return None

        _parse_rules(self)

    def __repr__(self):
        return self.workflow

    def __lt__(self, other):
        return self.name<other.name

In [9]:
wf_lines[0]

'ft{m<1347:R,m<1967:A,A}'

In [10]:
Workflow(wf_lines[0]).name

'ft'

In [11]:
workflows = [Workflow(wf) for wf in wf_lines]
workflows = {wf.name:wf for wf in workflows}

In [12]:
categories = 'xmas'
def get_part(line):
    ratings = [int(p.split('=')[1].split('}')[0]) for p in line.split(',')]
    return dict(zip(categories,ratings))

In [13]:
parts = [get_part(line) for line in parts_lines]

In [14]:
part = parts[0]
part

{'x': 238, 'm': 232, 'a': 127, 's': 30}

In [15]:
import operator

In [16]:
operation = {'<': operator.lt,
             '>': operator.gt
            }

In [17]:
def is_condition(rule, part):
    category = rule.category
    rule_value = rule.rating
    op = operation[rule.operation]
    part_rating = part[category]
    # print(part_rating,rule.operation,rule_value)
    return op(part_rating,rule_value)

In [18]:
start = workflows['in']
start

in{a<1943:sg,x<1758:tpb,spv}

In [19]:
start.rules, start.next_workflow

([a<1943:sg, x<1758:tpb], 'spv')

In [20]:
wf = workflows['rfg']
print(wf)

def get_next_workflow(wf, part):
    rs = wf.rules[::-1].copy()
    while len(rs) > 0:
        rule = rs.pop()
        # print(rule)
        if is_condition(rule, part):
            # print('passed')
            name = rule.next_workflow
            if (name=='A')|(name=='R'):
                # print(name)
                return name
            else:
                return workflows[name]          
        else:
            pass
            # print('failed, trying next rule')
    # print('ran out of rules')
    return wf.next_workflow
        

rfg{s<1129:ft,m>1514:mk,m>934:jpb,ggs}


In [21]:
get_next_workflow(wf,part)

ft{m<1347:R,m<1967:A,A}

In [22]:
def get_part_decision(part):
    # print(workflows['in'])
    next_wf = get_next_workflow(workflows['in'],part)
    while (next_wf!='A')&(next_wf!='R'):
        # print(f'part {part}: {next_wf}')
        try:
            next_wf = get_next_workflow(workflows[next_wf], part)
        except:
            next_wf = get_next_workflow(workflows[next_wf.name],part)
    return next_wf   

In [23]:
accepted = list()
for part in parts:
    decision = get_part_decision(part)
    if decision=='A':
        accepted.append(part)
len(accepted)

79

In [24]:
ratings = [sum(val for key, val in part.items()) for part in accepted]
len(ratings)

79

In [25]:
sum(ratings)

342650